# ABN Postcode Extractor

## Required Imports

In [5]:
import numpy as np
import pandas as pd
import pathlib
import xml.etree.ElementTree as ET
from zipfile import ZipFile
from urllib.request import urlretrieve
import glob

# Get List of ABNs

In [6]:
urlretrieve('https://data.gov.au/data/dataset/c2524c87-cea4-4636-acac-599a82048a26/resource/6eaf8203-a79b-43f0-9671-feeb1a44cbff/download/2021-22-corporate-report-of-entity-tax-information.xlsx', '2021-22-corporate-report-of-entity-tax-information.xlsx')

abn_list_df = (pd.read_excel('2021-22-corporate-report-of-entity-tax-information.xlsx', sheet_name='Income tax details', dtype={'ABN': str})
               .rename(columns={'Name': 'name',
                                'ABN': 'abn_tax',
                                'Total income $': 'total_income',
                                'Taxable income $': 'taxable_income',
                                'Tax payable $': 'tax_payable',
                                'Income year': 'income_year'}
                      )
               .query('not(abn_tax.isna()) & income_year == "2021-22"')
               ['abn_tax']
               .to_frame()
               .reset_index(drop=True)
          )

abn_list_df

,abn_tax
0,83114980880
1,16634403124
2,33104201014
3,95650096094
4,93641633141
...,...
2688,87609580690
2689,93107449534
2690,50139546428
2691,94156476425


## Get ABN Bulk Extract Files

In [7]:
abn_bulk_data_links = {
    'abn_extract_1.zip': 'https://data.gov.au/data/dataset/5bd7fcab-e315-42cb-8daf-50b7efc2027e/resource/0ae4d427-6fa8-4d40-8e76-c6909b5a071b/download/public_split_11_20.zip',
    'abn_extract_2.zip': 'https://data.gov.au/data/dataset/5bd7fcab-e315-42cb-8daf-50b7efc2027e/resource/635fcb95-7864-4509-9fa7-a62a6e32b62d/download/public_split_1_10.zip'
}

for file, link in abn_bulk_data_links.items():
    urlretrieve(link, file)
    with ZipFile(file, 'r') as zip:
        zip.extractall()
    pathlib.Path(file).unlink()

## Find Postcodes from ABR Bulk Extract

In [8]:
abn_info_dict = {'abn_abr': [],
                 'postcode': []
                }

xml_root = ET.parse('20241016_Public01.xml').getroot()

for record in xml_root.iter('ABN'):
    abn_info_dict['abn_abr'].append(record.text)

for record in xml_root.iter('Postcode'):
    abn_info_dict['postcode'].append(record.text)

abn_info_df = pd.DataFrame.from_dict(abn_info_dict)

del abn_info_dict

abn_merge = (abn_list_df
             .merge(abn_info_df, how='left', left_on='abn_tax', right_on='abn_abr')
             .reset_index(drop=True)
            )

del abn_list_df

abn_merge_success = (abn_merge
                     .query('not(abn_abr.isna())')
                     .reset_index(drop=True)
                    )

abn_merge_unsuccessful = (abn_merge
                          .query('abn_abr.isna()')
                          .drop(columns=['abn_abr', 'postcode'])
                          .reset_index(drop=True)
                         )
del abn_merge

for xml in glob.glob('*.xml'):
    abn_info_dict = {'abn_abr': [],
                     'postcode': []
                    }
    
    xml_root = ET.parse(xml).getroot()
    
    for record in xml_root.iter('ABN'):
        abn_info_dict['abn_abr'].append(record.text)
    
    for record in xml_root.iter('Postcode'):
        abn_info_dict['postcode'].append(record.text)
    
    abn_info_df = pd.DataFrame.from_dict(abn_info_dict)
    
    del abn_info_dict  
    
    abn_merge = (abn_merge_unsuccessful
                 .merge(abn_info_df, how='left', left_on='abn_tax', right_on='abn_abr')
                 .reset_index(drop=True)
                )
    
    abn_merge_success = (pd.concat([abn_merge_success, abn_merge.query('not(abn_abr.isna())')])
                         .reset_index(drop=True)
                        )

    if abn_merge.query('abn_abr.isna()').shape[0] == 0:
        break

    abn_merge_unsuccessful = (abn_merge
                              .query('abn_abr.isna()')
                              .drop(columns=['abn_abr', 'postcode'])
                              .reset_index(drop=True)
                             )
    del abn_merge

del abn_merge

abn_found_postcodes = (abn_merge_success
                       [['abn_tax', 'postcode']]
                       .query('not(postcode.isna())')
                       .rename(columns={'abn_tax': 'abn'})
                      )

abn_found_postcodes

,abn,postcode
0,12619244852,2000
1,11008871761,6007
2,13637089162,3000
3,15007596018,5000
4,15094531272,3205
...,...,...
2688,85108448228,3166
2689,84001657370,2060
2690,84004259527,3006
2691,85003550776,2178


## Output Corporate Tax Data with Postcodes & Final Cleanup

In [9]:
abn_found_postcode_tax_concordance = (pd.read_excel('2021-22-corporate-report-of-entity-tax-information.xlsx', sheet_name='Income tax details', dtype={'ABN': str})
                                      .rename(columns={'Name': 'name',
                                                       'ABN': 'abn',
                                                       'Total income $': 'total_income',
                                                       'Taxable income $': 'taxable_income',
                                                       'Tax payable $': 'tax_payable',
                                                       'Income year': 'income_year'}
                                              )
                                      .query('not(abn.isna()) & income_year == "2021-22"')
                                      .merge(abn_found_postcodes, how='left', on='abn')
                                      .query('not(postcode.isna())')
                                      .reset_index(drop=True)
                                     )

del abn_found_postcodes

abn_found_postcode_tax_concordance

,name,abn,total_income,taxable_income,tax_payable,income_year,postcode
0,1884 PTY LIMITED,83114980880,215297043,8124632.0,2367120.0,2021-22,2007
1,20 CASHEWS PTY LTD,16634403124,288582715,36977225.0,962761.0,2021-22,3000
2,21ST CENTURY RESORTS HOLDINGS PTY LIMITED,33104201014,255202722,19673962.0,5902189.0,2021-22,2065
3,29METALS LIMITED,95650096094,501817901,NaN,NaN,2021-22,3000
4,2SAN PTY LTD,93641633141,739184508,70223411.0,21067023.0,2021-22,2000
...,...,...,...,...,...,...,...
2685,ZFAAS HOLDINGS PTY LTD,87609580690,219495534,206108050.0,61832415.0,2021-22,4006
2686,ZIMMER AUSTRALIA HOLDING PTY LIMITED,93107449534,282546149,17665597.0,5299679.0,2021-22,2086
2687,ZIP CO LIMITED,50139546428,304646327,NaN,NaN,2021-22,2000
2688,ZOETIS AUSTRALIA PTY LTD,94156476425,469791424,26261399.0,7878420.0,2021-22,2138


In [10]:
pathlib.Path('2021-22-corporate-report-of-entity-tax-information.xlsx').unlink()

abn_found_postcode_tax_concordance.to_csv('2021-22_corporate_tax_transparency_postcode_data.csv', index=False)

for file in glob.glob('*.xml'):
    pathlib.Path(file).unlink()